In [1]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
def main():
    # 파일 불러오기
    movie_raw = pd.read_csv("data_ver4.csv", dtype={'관객수':np.str_, '스크린수':np.str_, '상영횟수':np.str_})
    movie_raw = movie_raw.loc[:, "영화명": "배우"]
    movie = movie_raw

    # 점유율, 제작사 열 삭제
    movie.drop(["제작사", "점유율"], axis=1, inplace=True)

    #  데이터 추출 후 Nan을 빈값으로 바꿔줌
    movie = movie.fillna('')

    # 엑셀파일 만들기
    outWorkbook = xlsxwriter.Workbook("Distributor"+ datetime.today().strftime("%Y%m%d%H%M%S")+".xlsx")
    worksheet = outWorkbook.add_worksheet("Distributor")

    #엑셀의 해더를 만든다.
    excel_export_head(worksheet)


    # 엑셀 1행부터 시작하기 위해 선언
    cnt = 1

    # 예외처리 후 프로그램이 돌다 죽을 경우를 대비해서 엑셀파일 생성 여부를 판단하는 구분자.
    excel_yn = False
    try:

        # CSV 파일에서 읽은 데이터 프레임을 행별로 뽑는다.
        for index,row in movie.iterrows():

            distribution = row['배급사']
            grade = row['등급']
            genre= row['장르']
            actor = row['배우']

            # 배급사, 등급, 장르, 배우가 모두 있는경우 크롤링 안함.
            if( len(distribution) > 0 and len(grade) > 0  and len(genre) > 0 and len(actor) > 0):
                movie_dic = {}
                movie_dic['배급사'] = distribution
                movie_dic['등급'] = grade
                movie_dic['장르'] = genre
                movie_dic['배우'] = actor

                print("전처리데이터:", cnt, row['영화명'], " -> ", movie_dic)
                print("--------------------------------------------------")
                # 엑셀파일 생성
                excel_export(worksheet, cnt, row, movie_dic)

            else:

                # 영화리스트에서 해당영화를 찾고 데이터 딕셔너리를 리턴한다.
                movie_dic = movie_search_list(row)

                # 받은 데이터 딕셔너리를 엑셀에 출력한다.
                excel_export(worksheet, cnt, row, movie_dic)
                print("크롤링데이터:",cnt , row['영화명']," -> " , movie_dic)
                print("--------------------------------------------------")

            # 다음행을 가르키기 위해 하나씩 더한다.
            cnt += 1
    except Exception as e:  # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
        cnt += 1 # 행번호 증가
        print("예외처리된 영화명 :", cnt, row['영화명'])
        print('예외가 발생했습니다. 예외처리 내용', e)
        print("--------------------------------------------------")
        excel_yn = True # 예외처리후 엑셀을 다시 닫는걸 방지.
        outWorkbook.close() # 엑셀 닫고 생성

    if(excel_yn == False):
        # 엑셀파일을 다 쓴후 닫는다.
        outWorkbook.close()

In [3]:
def excel_export_head(worksheet):

    worksheet.write(0, 0, "영화명")
    worksheet.write(0, 1, "개봉일")
    worksheet.write(0, 2, "매출액")
    worksheet.write(0, 3, "누적매출액")
    worksheet.write(0, 4, "관객수")
    worksheet.write(0, 5, "누적관객수")
    worksheet.write(0, 6, "스크린수")
    worksheet.write(0, 7, "상영횟수")
    worksheet.write(0, 8, "대표국적")
    worksheet.write(0, 9, "배급사")
    worksheet.write(0, 10, "등급")
    worksheet.write(0, 11, "장르")
    worksheet.write(0, 12, "감독")
    worksheet.write(0, 13, "배우")

In [4]:
# 엑셀 데이터 쓰기
def excel_export(worksheet,cnt, row,movie_dic):
    worksheet.write(cnt, 0, row['영화명'])
    worksheet.write(cnt, 1, row["개봉일"])
    worksheet.write(cnt, 2, row["매출액"])
    worksheet.write(cnt, 3, row["누적매출액"])
    worksheet.write(cnt, 4, row["관객수"])
    worksheet.write(cnt, 5, row["누적관객수"])
    worksheet.write(cnt, 6, row["스크린수"])
    worksheet.write(cnt, 7, row["상영횟수"])
    worksheet.write(cnt, 8, row["대표국적"])
    worksheet.write(cnt, 9, movie_dic["배급사"])
    worksheet.write(cnt, 10, movie_dic["등급"])
    worksheet.write(cnt, 11, movie_dic["장르"])
    worksheet.write(cnt, 12, row["감독"])
    worksheet.write(cnt, 13, movie_dic["배우"])

In [5]:
# 네이버 영화리스트 페이지에서 찾고자 하는 영화를 찾는다.
def movie_search_list(row):
    movie_dic = {}
    movie_dic['배급사'] = ""
    movie_dic['등급'] = ""
    movie_dic['장르'] = ""
    movie_dic['배우'] = ""
    movie_name = row['영화명']
    director_name = row["감독"]

    time.sleep(1)

    # 네이버 영화 페이지에서 영화명 검색
    url = 'https://movie.naver.com/movie/search/result.naver?section=movie&query=' + str(movie_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')

    # 검색되는 영화이름 모두 찾기
    # ul 태그의 클래스 search_list_1 찾음
    items = soup.find("ul", {"class": "search_list_1"})

    # 검색되는 영화가 있을 경우 확인
    if (items != None):
        # ul 태그 아래의 li 태그를 모두 찾음
        li_items = items.find_all('li')

        # 결과값을 담을 리스트 생성
        item_list = []
        for li_item in li_items:

            # 결과값을 담을 딕셔너리 생성
            item_dict = {}

            # 영화제목, URL, 감독 값을 태그에서 찾아 딕셔너리
            # 감독명과 타이틀에서 공백을 제거하고 리스트에 넣는다.
            name_tag = li_item.find('dt')
            name = name_tag.text
            if (name.find('(') > -1):
                name = ''.join(name[:name.find('(') - 1].split())  # 모든 공백제거
                item_dict['타이틀'] = name
            else:
                name = ''.join(name.split())  # 모든 공백제거
                item_dict['타이틀'] = name

            movie_url = name_tag.find('a')['href']
            item_dict['주소'] = movie_url
            dd_items = li_item.find_all("dd", {"class": "etc"})

            for dd_item in dd_items:
                dd_tag = dd_item.text

                if (dd_tag.find('감독') != -1):
                    if (dd_tag.find('|') > -1):
                        director_tag = dd_tag[:dd_tag.find('|')]
                        director = str(director_tag).replace('감독 : ', "") # 감독 단어제거
                        director = ''.join(director.split()) # 모든 공백제거
                        item_dict['감독'] = director
                    else:
                        director = dd_tag.replace('감독 : ', "")
                        director = ''.join(director.split())
                        item_dict['감독'] = director

            # 리스트에 딕셔너리를 넣는다
            item_list.append(item_dict)

        #  네이버에서 가져온 영화정보를 CSV 파일에서 가져온 값과 비교를 해서 정말 그 영화인지 찾는다.
        for naver_item in item_list:

            title = naver_item.get('타이틀')
            director = naver_item.get('감독')
            url = str(naver_item.get('주소')).replace('/movie/bi/mi/basic.naver?code=', '')

            director_name =  ''.join(str(director_name).split()) # 전처리 데이터의 감독명 공백제거
            movie_name = ''.join(str(movie_name).split()) # 전처리 데이터 영화이름 공백제거

            if(director != None):
                director_ratio = name_match_ratio(director,director_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                director_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%

            if(title != None):
                movie_name_ratio = name_match_ratio(title,movie_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                movie_name_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%


            if (director_ratio >= 60 and movie_name_ratio == 100):

                url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=' + str(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "lxml", from_encoding="utf-8")

                movie_dic['배급사'] = movie_distribution(soup) # 배급사 찾기함수
                movie_dic['등급'] = movie_grade(soup) # 등급찾기 함수
                movie_dic['장르'] = movie_genre(soup) # 장르찾기 함수
                movie_dic['배우'] = movie_actor(soup) # 배우찾기

                print(' -- 감독명 : ', director, "||", director_name, '-->  일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title,'-->  일치율: ' ,movie_name_ratio, "%")
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
            else:

                print(' -- 감독명 : ',director,"||",director_name, '--> 일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title ,'--> 일치율: ' ,movie_name_ratio, "%" )
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
                
    return movie_dic

In [6]:
# 두 단어간 일치 비율을 리턴한다.
def name_match_ratio(word1,word2):

    d1 = list(filter(None,list(word1))) # 공백문자제거
    d1 = list(set(d1).intersection(d1)) # 중복되는 글자 원소 한개만 남김
    d2 = list(filter(None,list(word2))) # 공백문자제거
    d2 = list(set(d2).intersection(d2)) # 중복되는 글자 원소 한개만 남김
    d3 = list(set(d1).intersection(d2)) # 서로 동일한 단어를 리스트화
    if(word1 == word2):
        ratio = 100 #두단어를 비교했는데 정확하게 일치하면 100%
    else:
        # 두단어를 비교했는데 같은 글자의 갯수를 찾고
        ratio = int(len(d3) / len(d1) * 100) # 두 리스트 원소갯수를 비교하여 백분율로 변환
        print(d1,"::",d2,"-->",d3)

    return ratio

In [7]:
# 배급사정보를 html에서 찾는다.
def movie_distribution(soup):
    distribution = ''

    items = soup.find("dl", {"class": "agency_name"})
    # 배급사 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 배급사 정보 체크
            if (i.text == '배급'):
                distribution = str(dd_items[idx].text).replace('\n', '').replace('\t','').replace('\r','')
    return distribution

In [8]:
# 등급을 html에서 찾는다.
def movie_grade(soup):
    grade = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 등급 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 등급 정보 체크
            if (i.text == '등급'):
                grad_line = str(dd_items[idx].text).replace('\n', '').replace('\r', '').replace('\t', '')
                if(grad_line.find('[해외]')> -1):
                    grade = grad_line[:grad_line.find('[해외]')].replace('[국내] ','')
                else:
                    grade = grad_line
    return grade

In [9]:
def movie_genre(soup):
    genre = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 장르 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 장르 정보 체크
            if (str(i.text).find('개요') > -1):
                dd_spans = dd_items[idx].find_all('span')
                genre = str(dd_spans[0].text).replace('\n', '').replace('\r', '').replace('\t', '').split(',')[0]
    return genre

In [10]:
def movie_actor(soup):
    actor = ''
    actor_namelist = []
    items = soup.find("ul", {"class": "lst_people"})
    # 배우 정보가 없을 경우 체크
    if (items != None):
        # 모든 배우이름을 찾는다.
        a_items = items.find_all('a', {"class": "k_name"})
        # 배우이름을 담을 리스트를 만든다.

        # 찾은 모든 배우를 찾아 리스트에 담는다.
        for idx, i in enumerate(a_items):
            # 배우 정보 체크
            actor_name = str(i.text)
            actor_namelist.append(actor_name)
            # 4번째부터는 안넣음.
            if(idx == 2):
                break
    if actor_namelist != None:
        actor = ",".join(actor_namelist)

    return actor

In [11]:
if __name__ == '__main__':
    main()

전처리데이터: 1 도둑들  ->  {'배급사': '(주)쇼박스', '등급': '15세이상관람가', '장르': '액션', '배우': '김윤석,김혜수,이정재,전지현,임달화,김해숙,오달수,김수현,이심결,증국상,서영주,장동,장현문,정윤헌,조상원,한규석,홍원기,김영민,최덕문,최진호,장준녕,예수정,김영웅,이은채,손병욱,이동진'}
--------------------------------------------------
전처리데이터: 2 광해, 왕이 된 남자  ->  {'배급사': '(주)씨제이이엔엠', '등급': '15세이상관람가', '장르': '사극', '배우': '이병헌,류승룡,한효주,장광,김인권,심은경,김명곤,서진원,장재현,정창국,조혜정,김남준,이수용,박윤호,김지수,박민규,박지아,송미정,양준모,원동연,김종구,주영호,이엘,전배수,이란희,최욱,이봉련,허성태,이양희,전국향,김승훈,권은수,승의열,김비비,한이진,이은정'}
--------------------------------------------------
전처리데이터: 3 트랜스포머 3  ->  {'배급사': '(주)씨제이이엔엠', '등급': '12세이상관람가', '장르': '액션', '배우': '조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,샤이아 라보프,패트릭 뎀시'}
--------------------------------------------------
전처리데이터: 4 미션임파서블:고스트프로토콜  ->  {'배급사': '(주)씨제이이엔엠', '등급': '15세이상관람가', '장르': '액션', '배우': '톰 크루즈,제레미 레너'}
--------------------------------------------------
전처리데이터: 5 최종병기 활  ->  {'배급사': '롯데쇼핑㈜롯데엔터테인먼트', '등급': '15세이상관람가', '장르': '사극', '배우': '박해일,류승룡,김무열,문채원,이한위,김구택,이경영,박기웅,이승준,이재구,박노식,이다윗,전민서,오타니 료헤이,강규영,김홍수,김길동,최선미,조우진

['힐', '드', '닉', '니', '아', '바'] :: ['드', '하', '클', '호', '인', '스', ',', '라', '프', '루', '태', '거'] --> ['드']
['리', '감', '2', '용', '석', '와', '들', '한', '빌', '녀'] :: ['리', '감', '용', 'D', '석', '와', '들', '3', '한', '빌', '녀'] --> ['리', '감', '용', '녀', '와', '들', '한', '빌', '석']
 -- 감독명 :  닉힐아드바니 || 라인하드클루스,호거태프 --> 일치율:  16 %
 -- 영화이름 :  빌리와용감한녀석들3D || 빌리와용감한녀석들2 --> 일치율:  90 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  라인하드클루스,호거태프 || 라인하드클루스,호거태프 -->  일치율:  100 %
 -- 영화이름 :  빌리와용감한녀석들3D || 빌리와용감한녀석들3D -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 188 빌리와 용감한 녀석들 3D  ->  {'배급사': 'SBS콘텐츠허브', '등급': '[국내] 전체 관람가', '장르': '애니메이션', '배우': '제임스 코든,짐 브로드벤트,빌리 비치'}
--------------------------------------------------
전처리데이터: 189 로렉스  ->  {'배급사': '유니버설픽쳐스인터내셔널 코리아(유)', '등급': '전체관람가', '장르': '애니메이션', '배우': '대니 드비토,잭 에프런,에드 헬름스,베티 화이트,롭 리글'}
--------------------------------------------------
전처리데이터: 190 위험한 관계  ->  {'배급사': '(주)씨제이이엔엠', '등급': '청소년관람불가', '장르': '멜로/로맨스', '배우': '장동건,장쯔이,장백지'}
------------

['노', '마', '니', '히', '리', '기', '요', '스', ',', '쿠', '시', '타', '사', '오', '카'] :: ['마', '니', '히', '기', '스', '쿠', '시', '사'] --> ['히', '마', '니', '기', '스', '쿠', '시', '사']
 -- 감독명 :  스기시마쿠니히사,오타카요시노리 || 스기시마쿠니히사 --> 일치율:  53 %
 -- 영화이름 :  극장판메탈베이블레이드VS태양작열의침략자솔블레이즈 || 극장판메탈베이블레이드VS태양작열의침략자솔블레이즈 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 306 극장판 메탈 베이블레이드 VS 태양 작열의 침략자 솔블레이즈  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
전처리데이터: 307 미트 페어런츠 3  ->  {'배급사': '(주)씨제이이엔엠', '등급': '15세이상관람가', '장르': '코미디', '배우': '더스틴 호프만,오웬 윌슨,벤 스틸러,로버트 드 니로'}
--------------------------------------------------
전처리데이터: 308 그을린 사랑  ->  {'배급사': '(주)티캐스트', '등급': '청소년관람불가', '장르': '드라마', '배우': '루브나 아자발'}
--------------------------------------------------
전처리데이터: 309 사랑한다, 사랑하지 않는다  ->  {'배급사': '(주)넥스트엔터테인먼트월드(NEW)', '등급': '15세이상관람가', '장르': '멜로/로맨스', '배우': '현빈,임수정,유진영,김중기,김혜옥'}
--------------------------------------------------
전처리데이터: 310 레지던트  ->  {'배급사': '(주

 -- 감독명 :  정길훈 || 정길훈 -->  일치율:  100 %
 -- 영화이름 :  엄마까투리 || 엄마까투리 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 428 엄마 까투리  ->  {'배급사': '(주)키노아이', '등급': '[국내] 전체 관람가', '장르': '애니메이션', '배우': '이소은,한신정,홍범기'}
--------------------------------------------------
전처리데이터: 429 드라이브  ->  {'배급사': '주식회사 풍경소리,판씨네마(주)', '등급': '청소년관람불가', '장르': '액션', '배우': '라이언 고슬링,캐리 뮬리건'}
--------------------------------------------------
전처리데이터: 430 상실의 시대  ->  {'배급사': '예지림엔터테인먼트', '등급': '청소년관람불가', '장르': '드라마', '배우': '키쿠치 린코,마츠야마 켄이치'}
--------------------------------------------------
전처리데이터: 431 토마스와 친구들 - 극장판 3  ->  {'배급사': '에이원엔터테인먼트', '등급': '전체관람가', '장르': '애니메이션', '배우': '지진희,마이클 브랜든'}
--------------------------------------------------
전처리데이터: 432 비기너스  ->  {'배급사': '유니버설픽쳐스인터내셔널 코리아(유)', '등급': '15세이상관람가', '장르': '드라마', '배우': '크리스토퍼 플러머,이완 맥그리거'}
--------------------------------------------------
전처리데이터: 433 터치  ->  {'배급사': '(주)팝엔터테인먼트', '등급': '청소년관람불가', '장르': '드라마', '배우': '유준상,김지영,임옥상,장주철,정석현,조영우,진영수,최

['도', '드', '캐', '빈', '널', '맥'] :: ['도', '드', '빈', '널', '케', '맥'] --> ['도', '드', '빈', '널', '맥']
 -- 감독명 :  캐빈맥도널드 || 케빈맥도널드 -->  일치율:  83 %
 -- 영화이름 :  말리 || 말리 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['췐', '조', '스', '이', 'H', '.'] :: ['도', '드', '빈', '널', '케', '맥'] --> []
 -- 감독명 :  조이스H.췐 || 케빈맥도널드 --> 일치율:  0 %
 -- 영화이름 :  말리 || 말리 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['시', '토', '반', '산'] :: ['도', '드', '빈', '널', '케', '맥'] --> []
 -- 감독명 :  산토시시반 || 케빈맥도널드 --> 일치율:  0 %
 -- 영화이름 :  말리 || 말리 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['클', '데', '프', '이', '랭', '빗'] :: ['도', '드', '빈', '널', '케', '맥'] --> []
['와', '리', '나', '말'] :: ['리', '말'] --> ['리', '말']
 -- 감독명 :  데이빗프랭클 || 케빈맥도널드 --> 일치율:  0 %
 -- 영화이름 :  말리 || 말리와나 --> 일치율:  50 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['소', '니', '월', '안'] :: ['도', '드', '빈', '널', '케', '맥'] --> []
['소', '리', '밥', '더', '7', '엑', '스', '말', '-'] :: ['리', '말'] --> ['리', '말']
 -- 감독명 :  안소니월 || 케빈맥도널드 --> 일치율:  0 %
 -- 영화이름 :  말리 || 밥말리

...

 -- 감독명 :  머빈르로이 || 머빈르로이 -->  일치율:  100 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['조', '지', '커', '큐'] :: ['로', '르', '빈', '이', '머'] --> []
 -- 감독명 :  조지큐커 || 머빈르로이 --> 일치율:  0 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['흥', '고', ',', '김', '형', '식'] :: ['로', '르', '빈', '이', '머'] --> []
 -- 감독명 :  고흥식,김형식 || 머빈르로이 --> 일치율:  0 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['로', '리', '치', '웰', '데', '이', '빗'] :: ['로', '르', '빈', '이', '머'] --> ['이', '로']
 -- 감독명 :  데이빗로웰리치 || 머빈르로이 --> 일치율:  28 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['고', '슬', '든', '러', '헤'] :: ['로', '르', '빈', '이', '머'] --> []
 -- 감독명 :  고든헤슬러 || 머빈르로이 --> 일치율:  0 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  None || 머빈르로이 --> 일치율:  0 %
 -- 영화이름 :  작은아씨들 || 작은아씨들 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 674 작은 아씨들  ->  {'배급사':

['마', '유', '쿠', '키', '사'] :: ['로', '조', '아', '슈', '건'] --> []
 -- 감독명 :  쿠사마유키 || 조슈아로건 --> 일치율:  0 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['혜', '송', '림'] :: ['로', '조', '아', '슈', '건'] --> []
 -- 감독명 :  송혜림 || 조슈아로건 --> 일치율:  0 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['영', '서', '권'] :: ['로', '조', '아', '슈', '건'] --> []
 -- 감독명 :  권서영 || 조슈아로건 --> 일치율:  0 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['빅', '로', '르', '파', '블', '주'] :: ['로', '조', '아', '슈', '건'] --> ['로']
 -- 감독명 :  주르파블로빅 || 조슈아로건 --> 일치율:  16 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['용', '김', '태'] :: ['로', '조', '아', '슈', '건'] --> []
 -- 감독명 :  김태용 || 조슈아로건 --> 일치율:  0 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['민', '식', '윤'] :: ['로', '조', '아', '슈', '건'] --> []
 -- 감독명 :  윤민식 || 조슈아로건 --> 일치율:  0 %
 -- 영화이름 :  피크닉 || 피크닉 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+

 -- 감독명 :  제롬로빈스,로버트와이즈 || 로버트와이즈,제롬로빈스 -->  일치율:  100 %
 -- 영화이름 :  웨스트사이드스토리 || 웨스트사이드스토리 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 713 웨스트 사이드 스토리  ->  {'배급사': '', '등급': '[국내] 전체 관람가', '장르': '뮤지컬', '배우': '나탈리 우드,리차드 베이머,러스 탬블린'}
--------------------------------------------------
전처리데이터: 714 러브 액츄얼리  ->  {'배급사': '(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마', '등급': '15세관람가,15세이상관람가', '장르': '멜로/로맨스', '배우': '휴 그랜트,콜린 퍼스,알란 릭맨,엠마 톰슨,리암 니슨,로라 린니,키이라 나이틀리,마틴 맥커천,빌 나이히'}
--------------------------------------------------
['민', '정', '김'] :: ['호', '변', '장'] --> []
 -- 감독명 :  김정민 || 변장호 --> 일치율:  0 %
 -- 영화이름 :  감자 || 감자 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['황', '길', '호'] :: ['호', '변', '장'] --> ['호']
 -- 감독명 :  황호길 || 변장호 --> 일치율:  33 %
 -- 영화이름 :  감자 || 감자 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['안', '르', '요', '스', '테'] :: ['호', '변', '장'] --> []
 -- 감독명 :  요안스테르 || 변장호 --> 일치율:  0 %
 -- 영화이름 :  감자 || 감자 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감

['리', '마', '우', '비', '스', '엔', '키', '린'] :: ['유', '형'] --> []
 -- 감독명 :  마리우스비엔린스키 || 유형 --> 일치율:  0 %
 -- 영화이름 :  리틀제이콥 || 리틀제이콥 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  유형 || 유형 -->  일치율:  100 %
 -- 영화이름 :  리틀제이콥 || 리틀제이콥 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 726 리틀 제이콥  ->  {'배급사': '', '등급': '[국내] 전체 관람가', '장르': '애니메이션', '배우': ''}
--------------------------------------------------
['리', '브', '스', '제', '판', '테'] :: ['상', '옥', '신'] --> []
 -- 감독명 :  스테판브리제 || 신상옥 --> 일치율:  0 %
 -- 영화이름 :  여자의일생 || 여자의일생 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['박', '태', '호'] :: ['상', '옥', '신'] --> []
 -- 감독명 :  박호태 || 신상옥 --> 일치율:  0 %
 -- 영화이름 :  여자의일생 || 여자의일생 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  신상옥 || 신상옥 -->  일치율:  100 %
 -- 영화이름 :  여자의일생 || 여자의일생 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['균', '경', '신'] :: ['상', '옥', '신'] --> ['신']
 -- 감독명 :  신경균 || 신상옥 --> 일치율:  33 %
 -- 영화이름 :  여자의일생 || 여자의일생 --> 일치율:  100 %
 -+-+-+

KeyboardInterrupt: 